In [1]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [4]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

**Question 1: Use yfinance to Extract Stock Data**

Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

In [6]:
tesla = yf.Ticker("TSLA")

In [7]:
tesla_data = tesla.history(period="max")

In [8]:
tesla_data.reset_index(inplace=True)

In [9]:
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


**Question 2: Use Webscraping to Extract Tesla Revenue Data**

In [10]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

In [11]:
html_data = requests.get(url).text

In [12]:
soup = BeautifulSoup(html_data, 'html.parser')

In [13]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

In [14]:
table = soup.find_all('tbody')[1]
for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) >= 2:
            date = cols[0].get_text(strip=True)
            revenue = cols[1].get_text(strip=True)
            tesla_revenue = pd.concat([
                tesla_revenue,
                pd.DataFrame([[date, revenue]], columns=['Date', 'Revenue'])
            ], ignore_index=True)

In [15]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(r',|\$', '', regex=True)

In [16]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [17]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


**Question 3: Use yfinance to Extract Stock Data**

In [18]:
GameStop = yf.Ticker("GME")

In [19]:
GME_data = GameStop.history(period="max")

In [20]:
GME_data.reset_index(inplace=True)

**Question 4: Use Webscraping to Extract GME Revenue Data**

In [21]:
url2="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

In [22]:
html_data_2 = requests.get(url2).text

In [23]:
soup = BeautifulSoup(html_data_2, 'html.parser')

In [24]:
GameStop_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

In [25]:
table = soup.find_all('tbody')[1]
for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) >= 2:
            date = cols[0].get_text(strip=True)
            revenue = cols[1].get_text(strip=True)
            GameStop_revenue = pd.concat([
                 GameStop_revenue,
                pd.DataFrame([[date, revenue]], columns=['Date', 'Revenue'])
            ], ignore_index=True)

In [26]:
GameStop_revenue["Revenue"] = GameStop_revenue['Revenue'].str.replace(r',|\$', "")

In [27]:
GameStop_revenue.dropna(inplace=True)

GameStop_revenue = GameStop_revenue[GameStop_revenue['Revenue'] != ""]

In [28]:
GameStop_revenue.tail()

,Date,Revenue
57,2006-01-31,"$1,667"
58,2005-10-31,$534
59,2005-07-31,$416
60,2005-04-30,$475
61,2005-01-31,$709


**Question 5: Plot Tesla Stock Graph**

In [29]:
tesla_data['Date'] = pd.to_datetime(tesla_data['Date'])
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'])

def make_graph(tesla_data, tesla_revenue, tesla):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = tesla_data[tesla_data.Date <= '2021-06-14']
    revenue_data_specific = tesla_revenue[tesla_revenue.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title="tesla,inc",
    xaxis_rangeslider_visible=True)
    fig.show()
make_graph(tesla_data, tesla_revenue, tesla)

**Question 6: Plot GameStop Stock Graph**

In [30]:
GME_data['Date'] = pd.to_datetime(GME_data['Date'])
GameStop_revenue['Date'] = pd.to_datetime(GameStop_revenue['Date'])
GameStop_revenue['Revenue'] = GameStop_revenue['Revenue'].replace(r'[\$,]', '', regex=True).astype(float)

def make_graph(GME_data, GameStop_revenue, GameStop):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = GME_data[GME_data.Date <= '2021-06-14']
    revenue_data_specific = GameStop_revenue[GameStop_revenue.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title="GameStop",
    xaxis_rangeslider_visible=True)
    fig.show()
make_graph (GME_data, GameStop_revenue, GameStop)